# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

设置embedding - zhipu

In [79]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

from dotenv import load_dotenv, find_dotenv
import os
# 从环境变量中加载你的 API_KEY
_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# zhipuai_api_key

# 加载数据库

import os
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores.chroma import Chroma
from zhipuai_embedding import ZhipuAIEmbeddings  # 假设这是正确的导入路径


_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../chroma-will'

# 创建嵌入模型
from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=zhipuai_api_key
)

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        embedding_function=zhipu_embed
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

向量数据库已成功加载。


In [80]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


In [81]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


In [84]:
question = "什么是机器学习?"
docs = vectordb.similarity_search(question,k=15)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：15


打印一下检索到的内容

In [85]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》其他常见神经网络式的解释式的解释式的解释式的解释深度学习什么是深度学习深度学习的起源
-----------------------------------------------------
检索到的第1个内容: 
 欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》目录第章绪论引言基本术语假设空间归纳偏好式和式的解释第章模型评估与选择经验误差与过拟合评估方法算法参数超参数与模型参数
-----------------------------------------------------
检索到的第2个内容: 
 欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》式的推导梯度下降法牛顿法式的解释式的推导式的推导线性判别分析式的推导
-----------------------------------------------------
检索到的第3个内容: 
 前言周志华老师的《机器学习》西瓜书是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能不太友好，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周老师之所以省去这些推导细节的真实原因是，他本尊认为理工科数学基础扎实点的大二下学生应该对西瓜书中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习。所以本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的理工科数学基础扎实点的大二下学生。使用说明南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第章和第章的公式强烈不建议深究，简单过一下即可，等你学得有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力争
-----------------------------------------------------
检索到的第4个内容: 
 式的解释多变量决策树图

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [86]:
from langchain_openai import ChatOpenAI
zhipuai_llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=zhipuai_api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

## 3. 构建检索问答链

prompts

In [87]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [88]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [89]:
question_1 = "什么是南瓜书？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
？

这是一本关于机器学习的辅助教材，旨在帮助读者理解周志华老师的《机器学习》（西瓜书）中的公式和概念。

这本书特别感谢了早期对南瓜书做出贡献的人。

读者可以通过扫描二维码加入南瓜书的交流群。

版权声明表示这本书遵循知识共享署名非商业性使用相同方式共享国际许可协议。

南瓜书建议读者在遇到难以理解的公式时，以西瓜书为主线，辅以南瓜书。

对于初学者，不建议深入探究西瓜书中的某些章节公式，可以先简单了解，以后再深入研究。

南瓜书力求以本科生数学基础的角度解释公式，并在必要时提供额外的资料。

如果南瓜书中有错误或缺少内容，作者鼓励读者积极反馈。

谢谢你的提问！


### 创建一个基于模板的检索链： BM25检索版本

### 创建一个基于模板的检索链： rerank检索版本

In [105]:
# 测试API

import cohere
# co = cohere.ClientV2()
co = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]
response = co.rerank(
    model="rerank-v3.5",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)
print(response)



id='27c295c4-f96b-4e63-889f-9707b246f31c' results=[RerankResponseResultsItem(document=None, index=3, relevance_score=0.8742601), RerankResponseResultsItem(document=None, index=0, relevance_score=0.17292508), RerankResponseResultsItem(document=None, index=4, relevance_score=0.10793502)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [109]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    zhipuai_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)


## 4.检索问答链效果测试

In [110]:
question_1 = "什么是南瓜书？"
question_2 = "严威是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [111]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
南瓜书是针对周志华老师《机器学习》西瓜书中一些难理解公式和推导细节的补充解析，是数学基础较弱的读者在自学时整理的笔记，旨在帮助读者更好地理解和学习机器学习相关公式。它建议与西瓜书结合使用，遇到难以推导或理解的公式时查阅。谢谢你的提问！


In [112]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道严威是谁，这个信息在提供的上下文中没有提及。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [18]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
zhipuai_llm.predict(prompt_template)

C:\Users\will\AppData\Local\Temp\ipykernel_9320\1697173039.py:5: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zhipuai_llm.predict(prompt_template)


'南瓜书（Pumpkin Book）通常不是一个广为人知的标准名词，它可能指代不同的内容，具体含义取决于上下文。以下是几种可能的解释：\n\n1. **儿童书籍**：可能是指一本以南瓜为主题的儿童读物，这样的书籍通常会在万圣节前后出现，讲述与南瓜有关的故事或活动。\n\n2. **食谱书**：也可能是一本介绍如何制作南瓜相关菜肴的烹饪书籍，例如南瓜派、南瓜汤等。\n\n3. **园艺指导书**：关于如何种植和培育南瓜的园艺指南。\n\n4. **艺术作品或手工书**：一些书籍可能会教人如何雕刻南瓜灯或制作与南瓜相关的手工艺品。\n\n5. **特定文化或传统的参考书籍**：在某些文化中，南瓜可能具有重要的象征意义，相关的书籍可能会探讨这些含义。\n\n6. **网络流行文化**：在现代互联网文化中，“南瓜书”可能是某个特定的梗或者某个社区内的特定称谓。\n\n如果您是在特定的上下文中遇到“南瓜书”这个词，提供更多的信息可以帮助给出更准确的解释。'

In [19]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
zhipuai_llm.predict(prompt_template)

'很抱歉，根据我截至2023的知识库，"严威"这个名字并不足以确定是指哪位具体的人物。严威可能是一个普通的姓名，没有在公开的知名人物数据库或新闻报道中找到与之相关的显著信息。\n\n如果严威是一个特定领域的专家、学者或者其他类型的人物，并且您正在寻找关于此人的具体信息，请提供更多的上下文或者详细背景，这样我可能能够提供更准确的帮助。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能-v2025

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_9320\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

v2024

In [31]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    zhipuai_llm,
    retriever=retriever,
    memory=memory
)
question = "什么是南瓜书？"
result = qa({"question": question})
print(result['answer'])

根据提供的信息，南瓜书似乎是一个作品的名字，但是具体的内容和性质没有详细说明。从信息中可以看出，有一些人担任了编委会的主编和编委，还有人对封面进行了设计。此外，还特别感谢了一些人在南瓜书最早期时所做出的贡献。如果需要了解南瓜书更具体的信息，可能需要扫描提供的二维码并回复关键词“南瓜书”，或者在网络中进行进一步的查询。我这里没有足够的信息来定义南瓜书的具体内容。


V2025

不带memory

In [32]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# compressor = CohereRerank(
#     client=cohere_client,
#     top_n=5  # 保留Top5相关文档
# )
compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
#                                        retriever=base_retriever,
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

qa_chain = RetrievalQA.from_chain_type(
    zhipuai_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])


大模型+知识库后回答 question_1 的结果：
南瓜书是对周志华老师的《机器学习》（西瓜书）中部分公式推导细节的补充解析，旨在帮助读者更好地理解西瓜书中的难点。它适合在阅读西瓜书时遇到难以推导或理解的公式时查阅。南瓜书是数学基础较弱的读者在学习过程中的笔记汇集。

谢谢你的提问！


新增MEMORY

In [ ]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=zhipuai_llm,
    retriever=compression_retriever,
    memory=memory,
    # return_source_documents=True,
    output_key="answer",  # 明确指定存储到内存的键
    combine_docs_chain_kwargs={  # 替代chain_type_kwargs
        "prompt": QA_CHAIN_PROMPT
    },
    verbose=True, # 独立传递verbose参数
)


questions = [
    "南瓜书的主要内容？", 
    "第三章提到的关键技术是什么？",  # 需记忆前一轮的"主要内容"
    # "请用表格总结这些技术的优缺点"  # 需合并多轮信息
    "请讲前两个问题的答案整理成表格"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))

In [ ]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

# qa_chain = ConversationalRetrievalChain.from_llm(
#     llm=zhipuai_llm.bind(max_length=300),  # 绑定输出长度限制
#     retriever=compression_retriever,
#     memory=memory,  # 注入记忆模块
#     return_source_documents=True,
#     chain_type_kwargs={
#         "prompt": QA_CHAIN_PROMPT,
#         "verbose": True  # 可查看详细处理过程
#     }
# )

questions = [
    "南瓜书的作者是谁？",  # 初始问题
    "他们的研究领域是什么？",  # 依赖前文"作者"信息
    "这本书的出版时间？"  # 依赖前文"书"实体
]

questions = [
    "南瓜书的主要内容？", 
    "第三章提到的关键技术是什么？",  # 需记忆前一轮的"主要内容"
    "请用表格总结这些技术的优缺点"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))

In [24]:
# from langchain.chains import ConversationalRetrievalChain

# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=base_retriever
# )

# qa_chain = ConversationalRetrievalChain(
#     zhipuai_llm,
#     retriever=compression_retriever,  # 替换为压缩检索器
#     return_source_documents=True,
#     chain_type_kwargs={
#         "prompt": QA_CHAIN_PROMPT,
#         # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
#     }
# )

# question = "什么是南瓜书？"
# result = qa({"question": question})
# print(result['answer'])

C:\Users\will\AppData\Local\Temp\ipykernel_9320\2072884007.py:2: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~create_history_aware_retriever together with create_retrieval_chain (see example in docstring)` instead.
  qa_chain = ConversationalRetrievalChain(


TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
question = "南瓜书包含哪些内容？"
result = qa({"question": question})
print(result['answer'])

可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。

## 5. 添加历史对话的记忆功能 - v2024

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

/var/folders/4f/lx3tlt3d6fz2bys8ct46fzxh0000gn/T/ipykernel_90332/2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [16]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "什么是南瓜书？"
result = qa({"question": question})
print(result['answer'])

南瓜书是根据提供的上下文看起来是一本关于机器学习的书籍，名为《机器学习公式详解》。这本书由一个编委会负责，其中包括主编和编委，并且有一个封面设计团队。书中包含了机器学习的基本概念、模型评估与选择等内容，并且力图以本科数学基础的视角对公式进行解析和推导。此外，南瓜书还提供了配套的视频教程和在线阅读地址，以及通过GitHub平台收集读者反馈的方式。


然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [17]:
question = "南瓜书包含哪些内容？"
result = qa({"question": question})
print(result['answer'])

南瓜书主要包含以下内容和特点：

1. 以本科数学基础的视角对机器学习中的公式进行解析和推导。
2. 针对超纲的数学知识，通常以附录和参考文献的形式给出，方便感兴趣的同学深入学习。
3. 提供了一个GitHub的Issues页面（地址：https://github.com/datawhalechina/pumpkin-book/issues），供读者反馈错误或提出希望补充的公式编号。
4. 对于反馈，编委会承诺通常在24小时内回复。
5. 提供了微信联系方式（微信号：at-Sm1les），以便在超过24小时未回复的情况下与编委会联系。
6. 配套视频教程，可以在Bilibili上观看（地址：https://www.bilibili.com/video/BV1Mh411e7VU）。
7. 提供在线阅读地址（https://datawhalechina.github.io/pumpkin-book），仅供第1版。
8. 最新版PDF可以在GitHub的发布页面上获取（地址：https://github.com/datawhalechina/pumpkin-book/releases）。
9. 包含了从绪论到模型评估与选择等章节，具体内容涵盖引言、基本术语、假设空间、归纳偏好、经验误差与过拟合、评估方法以及算法参数（超参数）与模型参数等。

以上内容摘自提供的文本，具体书中内容可能更加丰富。


可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。